In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

#### Read training data from csv files

In [3]:
Train     = pd.read_csv('01train_scaled.csv', index_col=0, parse_dates=[8])
Test      = pd.read_csv('01test_scaled.csv', index_col=0, parse_dates=[8])

In [4]:
Test

,f1,f2,f3,f4,f5,f6,f7,Timestamp
0,0.289532,0.277328,0.226415,0.259574,0.204117,0.147727,0.150000,2019-06-03 00:00:00
1,0.258352,0.230769,0.277358,0.231915,0.198971,0.168561,0.160870,2019-06-03 00:05:00
2,0.211581,0.208502,0.232075,0.242553,0.202401,0.195076,0.213043,2019-06-03 00:10:00
3,0.242762,0.232794,0.213208,0.221277,0.176672,0.178030,0.195652,2019-06-03 00:15:00
4,0.238307,0.226721,0.235849,0.253191,0.192110,0.126894,0.128261,2019-06-03 00:20:00
...,...,...,...,...,...,...,...,...
4027,0.282851,0.253036,0.264151,0.302128,0.233276,0.196970,0.169565,2019-06-16 23:35:00
4028,0.262806,0.222672,0.243396,0.246809,0.166381,0.174242,0.180435,2019-06-16 23:40:00
4029,0.204900,0.188259,0.213208,0.270213,0.221269,0.166667,0.171739,2019-06-16 23:45:00
4030,0.273942,0.246964,0.247170,0.219149,0.183533,0.195076,0.193478,2019-06-16 23:50:00


In [5]:
type(Test.iloc[0]['Timestamp'])

pandas._libs.tslibs.timestamps.Timestamp

In [6]:
train = Train.drop('Timestamp', axis=1)
test = Test.drop('Timestamp', axis=1)

# 5-minute (1 step ahead) foreast

## Train

In [7]:
length = 15
step = 1
k = -1

a = np.zeros((15, 7))
b = np.zeros((1,))
x = np.zeros((15, 7))
y = np.zeros((1,))


for i in range(length, (len(train)-step+1)):
    b[0] = train["f5"][i+step-1]
    k = k+1
    for j in range(length):
        if j+k <= (len(train) + length - step):
            a[j] = train.iloc[j+k]
            
    x = np.append(x, a, axis=0)
    y = np.append(y, b, axis=0)
    
x = np.delete(x, np.s_[0:15], 0).reshape(-1, 15, 7)
y = np.delete(y, np.s_[0], 0)

In [8]:
x.shape

(31089, 15, 7)

In [9]:
y.shape

(31089,)

In [10]:
train.tail(16)

,f1,f2,f3,f4,f5,f6,f7
31088,0.483296,0.417004,0.390566,0.463830,0.421955,0.257576,0.282609
31089,0.374165,0.366397,0.339623,0.351064,0.308748,0.333333,0.319565
31090,0.489978,0.443320,0.445283,0.414894,0.387650,0.304924,0.276087
31091,0.416481,0.410931,0.433962,0.376596,0.346484,0.337121,0.336957
31092,0.383073,0.392713,0.373585,0.363830,0.336192,0.284091,0.276087
31093,0.351893,0.319838,0.341509,0.334043,0.300172,0.231061,0.267391
31094,0.280624,0.329960,0.318868,0.291489,0.252144,0.225379,0.230435
31095,0.298441,0.295547,0.316981,0.370213,0.336192,0.251894,0.263043
31096,0.378619,0.334008,0.339623,0.325532,0.295026,0.214015,0.226087
31097,0.293987,0.269231,0.301887,0.291489,0.253859,0.232955,0.239130


In [11]:
x[-1]

array([[0.48329621, 0.41700405, 0.39056604, 0.46382979, 0.4219554 ,
        0.25757576, 0.2826087 ],
       [0.37416481, 0.36639676, 0.33962264, 0.35106383, 0.30874786,
        0.33333333, 0.31956522],
       [0.48997773, 0.44331984, 0.44528302, 0.41489362, 0.38765009,
        0.30492424, 0.27608696],
       [0.41648107, 0.41093117, 0.43396226, 0.37659574, 0.3464837 ,
        0.33712121, 0.33695652],
       [0.3830735 , 0.39271255, 0.37358491, 0.36382979, 0.33619211,
        0.28409091, 0.27608696],
       [0.3518931 , 0.31983806, 0.34150943, 0.33404255, 0.30017153,
        0.23106061, 0.2673913 ],
       [0.28062361, 0.32995951, 0.31886792, 0.29148936, 0.25214408,
        0.22537879, 0.23043478],
       [0.29844098, 0.29554656, 0.31698113, 0.37021277, 0.33619211,
        0.25189394, 0.26304348],
       [0.37861915, 0.3340081 , 0.33962264, 0.32553191, 0.29502573,
        0.21401515, 0.22608696],
       [0.29398664, 0.26923077, 0.30188679, 0.29148936, 0.25385935,
        0.23295455, 0.2

In [12]:
# Checked as correct

In [13]:
# Save 

np.savez_compressed("Deep_train_5", x=x , y=y)

## Test

In [14]:
length = 15
step = 1
k = -1

a = np.zeros((15, 7))
b = np.zeros((1,))
x = np.zeros((15, 7))
y = np.zeros((1,))


for i in range(length, (len(test)-step+1)):
    b[0] = test["f5"][i+step-1]
    k = k+1
    for j in range(length):
        if j+k <= (len(test) + length - step):
            a[j] = test.iloc[j+k]
            
    x = np.append(x, a, axis=0)
    y = np.append(y, b, axis=0)
    
x = np.delete(x, np.s_[0:15], 0).reshape(-1, 15, 7)
y = np.delete(y, np.s_[0], 0)

In [15]:
x.shape

(4017, 15, 7)

In [16]:
y.shape

(4017,)

In [17]:
# Checked as correct

In [18]:
# Save 

np.savez_compressed("Deep_test_5", x=x , y=y)

# 15-minute (3 step ahead) forecast

## Train

In [19]:
length = 15
step = 3
k = -1

a = np.zeros((15, 7))
b = np.zeros((1,))
x = np.zeros((15, 7))
y = np.zeros((1,))


for i in range(length, (len(train)-step+1)):
    b[0] = train["f5"][i+step-1]
    k = k+1
    for j in range(length):
        if j+k <= (len(train) + length - step):
            a[j] = train.iloc[j+k]
            
    x = np.append(x, a, axis=0)
    y = np.append(y, b, axis=0)
    
x = np.delete(x, np.s_[0:15], 0).reshape(-1, 15, 7)
y = np.delete(y, np.s_[0], 0)

In [20]:
x.shape

(31087, 15, 7)

In [21]:
y.shape

(31087,)

In [22]:
# Checked as correct

In [23]:
# Save 

np.savez_compressed("Deep_train_15", x=x , y=y)

## Test

In [24]:
length = 15
step = 3
k = -1

a = np.zeros((15, 7))
b = np.zeros((1,))
x = np.zeros((15, 7))
y = np.zeros((1,))


for i in range(length, (len(test)-step+1)):
    b[0] = test["f5"][i+step-1]
    k = k+1
    for j in range(length):
        if j+k <= (len(test) + length - step):
            a[j] = test.iloc[j+k]
            
    x = np.append(x, a, axis=0)
    y = np.append(y, b, axis=0)
    
x = np.delete(x, np.s_[0:15], 0).reshape(-1, 15, 7)
y = np.delete(y, np.s_[0], 0)

In [25]:
x.shape

(4015, 15, 7)

In [26]:
y.shape

(4015,)

In [27]:
# Checked as correct

In [28]:
# Save 

np.savez_compressed("Deep_test_15", x=x , y=y)

# 30-minute (6 step ahead) forecast

## Train

In [29]:
length = 15
step = 6
k = -1

a = np.zeros((15, 7))
b = np.zeros((1,))
x = np.zeros((15, 7))
y = np.zeros((1,))


for i in range(length, (len(train)-step+1)):
    b[0] = train["f5"][i+step-1]
    k = k+1
    for j in range(length):
        if j+k <= (len(train) + length - step):
            a[j] = train.iloc[j+k]
            
    x = np.append(x, a, axis=0)
    y = np.append(y, b, axis=0)
    
x = np.delete(x, np.s_[0:15], 0).reshape(-1, 15, 7)
y = np.delete(y, np.s_[0], 0)

In [30]:
x.shape

(31084, 15, 7)

In [31]:
y.shape

(31084,)

In [32]:
# Checked as correct

In [33]:
# Save 

np.savez_compressed("Deep_train_30", x=x , y=y)

## Test

In [34]:
length = 15
step = 6
k = -1

a = np.zeros((15, 7))
b = np.zeros((1,))
x = np.zeros((15, 7))
y = np.zeros((1,))


for i in range(length, (len(test)-step+1)):
    b[0] = test["f5"][i+step-1]
    k = k+1
    for j in range(length):
        if j+k <= (len(test) + length - step):
            a[j] = test.iloc[j+k]
            
    x = np.append(x, a, axis=0)
    y = np.append(y, b, axis=0)
    
x = np.delete(x, np.s_[0:15], 0).reshape(-1, 15, 7)
y = np.delete(y, np.s_[0], 0)

In [35]:
x.shape

(4012, 15, 7)

In [36]:
y.shape

(4012,)

In [37]:
# Checked as correct

In [38]:
# Save 

np.savez_compressed("Deep_test_30", x=x , y=y)

# 60-minute (12 step ahead) forecast

## Train

In [39]:
length = 15
step = 12
k = -1

a = np.zeros((15, 7))
b = np.zeros((1,))
x = np.zeros((15, 7))
y = np.zeros((1,))


for i in range(length, (len(train)-step+1)):
    b[0] = train["f5"][i+step-1]
    k = k+1
    for j in range(length):
        if j+k <= (len(train) + length - step):
            a[j] = train.iloc[j+k]
            
    x = np.append(x, a, axis=0)
    y = np.append(y, b, axis=0)
    
x = np.delete(x, np.s_[0:15], 0).reshape(-1, 15, 7)
y = np.delete(y, np.s_[0], 0)

In [40]:
x.shape

(31078, 15, 7)

In [41]:
y.shape

(31078,)

In [42]:
# Checked as correct

In [43]:
# Save 

np.savez_compressed("Deep_train_60", x=x , y=y)

## Test

In [44]:
length = 15
step = 12
k = -1

a = np.zeros((15, 7))
b = np.zeros((1,))
x = np.zeros((15, 7))
y = np.zeros((1,))


for i in range(length, (len(test)-step+1)):
    b[0] = test["f5"][i+step-1]
    k = k+1
    for j in range(length):
        if j+k <= (len(test) + length - step):
            a[j] = test.iloc[j+k]
            
    x = np.append(x, a, axis=0)
    y = np.append(y, b, axis=0)
    
x = np.delete(x, np.s_[0:15], 0).reshape(-1, 15, 7)
y = np.delete(y, np.s_[0], 0)

In [45]:
x.shape

(4006, 15, 7)

In [46]:
y.shape

(4006,)

In [47]:
# Checked as correct

In [48]:
# Save 

np.savez_compressed("Deep_test_60", x=x , y=y)